In [16]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan

from datasets import load_dataset
import torch

from IPython.display import Audio
import soundfile as sf

In [2]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [3]:
inputs = processor(text="Don't count the days, make the days count.", return_tensors="pt")

In [5]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset cmu-arctic-xvectors downloaded and prepared to /export/home/lium/bdos/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.


In [7]:
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [8]:
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

In [10]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [11]:
with torch.no_grad():
    speech = vocoder(spectrogram)

In [13]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [17]:
Audio(speech, rate=16000)

In [15]:
#sf.write("tts_example.wav", speech.numpy(), samplerate=16000)